In [1]:
using Rx
using ReactiveMP
using Distributions

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1273
┌ Warning: Package ReactiveMP does not have Rx in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rx as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rx into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
struct NetworkDataActor <: Actor{Float64}
    inference_actor
end

function Rx.on_next!(actor::NetworkDataActor, data::Float64)
    next!(actor.inference_actor.y.values, data)
    next!(actor.inference_actor.mean.values, actor.inference_actor.m_p)
    next!(actor.inference_actor.variance.values, actor.inference_actor.v_p)
end

Rx.on_error!(actor::NetworkDataActor, err) = error(err)
Rx.on_complete!(actor::NetworkDataActor)   = "NetworkDataActor completed"

In [3]:
mutable struct InferenceActor <: Actor{AbstractMessage}
    computed   :: Vector{AbstractMessage}
    
    y          :: ObservedVariable
    mean       :: EstimatedVariable
    variance   :: EstimatedVariable
    
    m_p :: Float64
    v_p :: Float64
    
    InferenceActor(y::ObservedVariable, mean::EstimatedVariable, variance::EstimatedVariable) = begin
        return new(Vector{AbstractMessage}(), y, mean, variance, 0.0, 1000.0)
    end
end

function Rx.on_next!(actor::InferenceActor, data::AbstractMessage) 
    m = mean(data.distribution)
    v = var(data.distribution)
    
    actor.m_p = m
    actor.v_p = v
    
    push!(actor.computed, data)
    
    println(data)
end

Rx.on_error!(actor::InferenceActor, err) = error(err)
Rx.on_complete!(actor::InferenceActor)   = "InferenceActor completed"

In [4]:
x_prev_add   = addition_node("x_prev_add", StochasticMessage{Normal{Float64}}, DeterministicMessage, StochasticMessage{Normal{Float64}});
add_1        = constant_variable("add_1", 1.0, x_prev_add.in2);

x_prev_prior = gaussian_mean_variance("x_prev_prior");
x_prev_m     = estimated_variable("x_prev_m", x_prev_prior.mean);
x_prev_v     = estimated_variable("x_prev_v", x_prev_prior.variance);

x_prev = random_variable("x_prev", x_prev_prior.value, x_prev_add.in1);

noise_node     = gaussian_mean_variance("noise_node");
noise_mean     = constant_variable("noise_mean", 0.0, noise_node.mean);
noise_variance = constant_variable("noise_variance", 200.0, noise_node.variance);

add_x_and_noise = addition_node("add_x_and_noise", StochasticMessage{Normal{Float64}}, StochasticMessage{Normal{Float64}}, DeterministicMessage);

x = random_variable("x", x_prev_add.out, add_x_and_noise.in1);
n = random_variable("n", noise_node.value, add_x_and_noise.in2);
y = observed_variable("y", add_x_and_noise.out);

In [5]:
data = LocalNetworkObservable{Float64}(9013);

inference_actor = InferenceActor(y, x_prev_m, x_prev_v);
data_actor      = NetworkDataActor(inference_actor);

subject = Subject{Float64}()

inference_subscription = subscribe!(inference(x), inference_actor);
data_subscription      = subscribe!(data, subject);
subject_subscription   = subscribe!(subject, data_actor)

SubjectSubscription()

StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=7.612077529121848, σ=12.909944487358057))
StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=3.567711793514606, σ=9.534625892455924))
StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=-1.5203491112565417, σ=7.905694150420949))
StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=2.5911727929234125, σ=6.900655593423543))
StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=2.352621754693063, σ=6.201736729460423))
StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=0.11756848621170314, σ=5.679618342470648))
StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=3.0859979442256438, σ=5.2704627669473))
StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=5.693672573076477, σ=4.938647983247948))
StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=7.21859748318627, σ=4.66252404120157))
StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=9.71483434355701, σ=4.428074427700477))
StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=10.81

In [6]:
unsubscribe!(data_subscription)
unsubscribe!(inference_subscription)
unsubscribe!(subject_subscription)